In [1]:
import pandas as pd
import gzip
import sweetviz as sv
import os
import glob

/Users/camillecu/Downloads/KUL/MDA/project/waffle/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# import pit, mug and amb

In [4]:
file_path = "/Users/camillecu/Downloads/KUL/MDA/project/dataset/"

# Load the data from the parquet file
pit_loc = pd.read_parquet(file_path + "pit_locations.parquet.gzip")
amb = pd.read_parquet(file_path + "ambulance_locations.parquet.gzip")
mug = pd.read_parquet(file_path + "mug_locations.parquet.gzip")


# PIT - generate longitude and latitude

In [13]:
pit_loc.head()

,unit,campus,province,region,unit_id,ambucode,ambusitecode,link
0,"100001: PIT Rumst, AZ Rivierland",Ziekenhuis/Hôpital: 104 - AZ RIVIERENLAND --- ...,Antwerpen/Anvers,Vlaanderen/Flandre,PARUMS01A,191,19101,None
1,"102001: PIT Deinze, St Vincentius",Ziekenhuis/Hôpital: 134 - AZ SINT-VINCENTIUS -...,Oost-Vlaanderen/Flandre orientale,Vlaanderen/Flandre,PODEIN01A,413,41301,None
2,102101: PIT Stuivenberg,Ziekenhuis/Hôpital: 009 - ZIEKENHUISNETWERK AN...,Antwerpen/Anvers,Vlaanderen/Flandre,PAANTW01A,117,11704,None
3,"110001: PIT Lier, Heilig Hart",Ziekenhuis/Hôpital: 097 - HEILIG HART ZIEKENHU...,Antwerpen/Anvers,Vlaanderen/Flandre,PALIER01A,171,17101,None
4,113001: PIT HEILIG HART ZIEKENHUIS,Ziekenhuis/Hôpital: 102 - HEILIG HART ZIEKENHU...,Antwerpen/Anvers,Vlaanderen/Flandre,PAMOL_01A,190,19001,None


In [14]:
num_rows = pit_loc.shape[0]
print("Number of rows in pit dataset:", num_rows)

Number of rows in pit dataset: 24


In [5]:
# Specify the file path
file_path = "/Users/camillecu/Downloads/KUL/MDA/project/dataset/variables_ambureg.xlsx"

# Read the xlsx file into a DataFrame
ambureg = pd.read_excel(file_path, header=0)

# Display the DataFrame
ambureg.head()

,old ambulance code,ambulance code,site number,y,x,address,locality,ZIP,Province
0,167,35,3511,51.119490,4.667580,Pastorijstraat 25,BERLAAR,2590.0,Antwerpen
1,255,35,3522,50.743246,4.241075,Brusselsesteenweg 183,HALLE,1500.0,Vlaams-Brabant
2,315,35,3531,51.002498,3.322729,Bruggestraat 84,TIELT,8700.0,West-Vlaanderen
3,735,35,3571,51.248576,5.525886,Steenstraat 5,HAMONT-ACHEL,3930.0,Limburg
4,225,36,3621,50.660384,4.567357,Rue De Franquenies 10,OTTIGNIES,1341.0,Brabant Wallon


In [6]:
# merge x, y, address from ambureg to pit_loc where ambusitecode is the same as site number in ambureg
# only keep the columns x, y, address from ambureg
pit_loc = pd.merge(pit_loc, ambureg, left_on='ambusitecode', right_on='site number', how='left')

# only keep the columns that are needed: unit, campus, province, region, unit_id, ambucode, ambusitecode, x, y, address
pit_loc = pit_loc[['unit', 'campus', 'province', 'region', 'ambusitecode', 'x', 'y', 'address', 'ZIP']]

# Change column names from 'x' to 'latitude' and 'y' to 'longitude'
pit_loc = pit_loc.rename(columns={'x': 'latitude', 'y': 'longitude'})


In [7]:
# find duplicated value in pit_loc based on ambusitecode column
pit_loc[pit_loc.duplicated(['ambusitecode'])]


,unit,campus,province,region,ambusitecode,latitude,longitude,address,ZIP


In [8]:
# remove string 'Ziekenhuis/Hôpital: ' from campus column
pit_loc['campus'] = pit_loc['campus'].str.replace('Ziekenhuis/Hôpital: ', '')
# remove the first 6 characters from the campus column
pit_loc['campus'] = pit_loc['campus'].str[6:]
# remove string like '--- Campus/Site:' from campus column
pit_loc['campus'] = pit_loc['campus'].str.replace('--- Campus/Site:', '')

# remove 4 digits number from campus column
pit_loc['campus'] = pit_loc['campus'].str.replace('\d{4}', '', regex=True)

In [10]:
# seperate the campus column into 2 columns if there is a space followed by  a '-' character
pit_loc[['campus', 'site']] = pit_loc['campus'].str.split(' - ', 1, expand=True)


TypeError: StringMethods.split() takes from 1 to 2 positional arguments but 3 positional arguments (and 1 keyword-only argument) were given

In [11]:
pit = pit_loc.copy()

In [12]:
# Create a new column containing information on number,postal code and province of an address 
pit['unique_address'] = pit['address'].str.cat(others=pit[["region","province"]], sep=',',na_rep='')

In [13]:
# fill the missing values in longitude and latitude column with the value from the campus column using open street map
from geopy.geocoders import Nominatim
geocoder = Nominatim(user_agent="my_geocoder")
def get_coordinates(unique_address):
    try:
        location = geocoder.geocode(unique_address)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except (GeocoderTimedOut, GeocoderServiceError):
        return None, None


In [14]:
# Function to fill missing longitude and latitude using 'campus' values
def fill_missing_coordinates(row):
    if pd.isna(row['longitude']) or pd.isna(row['latitude']):
        latitude, longitude = get_coordinates(row['unique_address'])
        if pd.isna(row['longitude']):
            row['longitude'] = longitude
        if pd.isna(row['latitude']):
            row['latitude'] = latitude
    return row

# Apply the function to fill missing values
pit = pit.apply(fill_missing_coordinates, axis=1)

In [15]:
# print rows with missing values in longitude and latitude column
pit[pit['longitude'].isnull() | pit['latitude'].isnull()]

,unit,campus,province,region,ambusitecode,latitude,longitude,address,ZIP,unique_address


## MUG - generate longitude and latitude

In [17]:
mug.head()

,hospital_id,mug_id,campus_id,name_hospital,name_campus,address_campus,postal_code,municipality,region,province
0,9,102000,6230,ZIEKENHUISNETWERK ANTWERPEN (ZNA),ZNA CADIX,KEMPENSTRAAT 100,2030,ANTWERPEN,V,Antwerpen
1,99,102000,2020,GZA- ZIEKENHUIZEN,SINT-VINCENTIUS,ST.-VINCENTIUSSTRAAT 20,2018,ANTWERPEN,V,Antwerpen
2,682,102000,1210,AZ MONICA,DEURNE,FLORENT PAUWELSLEI 1,2100,DEURNE,V,Antwerpen
3,9,103000,2000,ZIEKENHUISNETWERK ANTWERPEN (ZNA),ZNA MIDDELHEIM,LINDENDREEF 1,2020,ANTWERPEN,V,Antwerpen
4,99,103000,1290,GZA- ZIEKENHUIZEN,SINT- AUGUSTINUS,OOSTERVELDLAAN 24,2610,WILRIJK,V,Antwerpen


In [18]:
# print missing values in the postal_code columns
mug[mug['postal_code'].isnull()]

,hospital_id,mug_id,campus_id,name_hospital,name_campus,address_campus,postal_code,municipality,region,province


In [19]:
mug[mug.duplicated(['mug_id'])]

,hospital_id,mug_id,campus_id,name_hospital,name_campus,address_campus,postal_code,municipality,region,province
1,99,102000,2020,GZA- ZIEKENHUIZEN,SINT-VINCENTIUS,ST.-VINCENTIUSSTRAAT 20,2018,ANTWERPEN,V,Antwerpen
2,682,102000,1210,AZ MONICA,DEURNE,FLORENT PAUWELSLEI 1,2100,DEURNE,V,Antwerpen
4,99,103000,1290,GZA- ZIEKENHUIZEN,SINT- AUGUSTINUS,OOSTERVELDLAAN 24,2610,WILRIJK,V,Antwerpen
6,710,104000,3450,AZ KLINA,None,AUGUSTIJNSLEI 100,2930,BRASSCHAAT,V,Antwerpen
17,713,202000,1610,AZ JAN PALFIJN,None,WATERSPORTLAAN 5,9000,GENT,V,Oost-Vlaanderen
21,176,205000,1800,A.S.Z. - AUTONOME VERZORGINGSINSTELLING,A.S.Z. AALST,MERESTRAAT 80,9300,AALST,V,Oost-Vlaanderen
56,332,528000,6140,CHIREC - DELTA,DELTA,TRIOMFLAAN 201,1160,OUDERGEM,B,Brussels Hoofdstedelijk Gewest
62,266,601000,2710,GROUPE JOLIMONT,HOPITAL DE MONS,AV. B. DE CONSTANTINOPLE 5,7000,MONS,W,Henegouwen
65,146,603000,3200,GROUPE JOLIMONT,HOPITAL DE JOLIMONT,RUE FERRER 159,7100,HAINE-SAINT-PAUL,W,Henegouwen
67,266,604000,2720,GROUPE JOLIMONT,HOPITAL DE WARQUIGNIES,RUE DES CHAUFOURS 27,7300,BOUSSU,W,Henegouwen


In [20]:
# Create a new column containing information on address_campus,postal code, municipality and  province 
mug['unique_address'] = mug["address_campus"].str.cat(others=mug[["municipality","province"]], sep=',',na_rep='')    

import numpy as np
# create two new columns: longitude and latitude with missing value
mug['longitude'] = np.nan  # Initialize with NaN (missing value)
mug['latitude'] = np.nan  # Initialize with NaN (missing value)

In [21]:
# fill the missing values in longitude and latitude column with the value from the campus column using open street map
from geopy.geocoders import Nominatim
geocoder = Nominatim(user_agent="my_geocoder")
def get_coordinates(unique_address):
    try:
        location = geocoder.geocode(unique_address)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except (GeocoderTimedOut, GeocoderServiceError):
        return None, None


In [22]:
# Function to fill missing longitude and latitude using 'campus' values
def fill_missing_coordinates(row):
    if pd.isna(row['longitude']) or pd.isna(row['latitude']):
        latitude, longitude = get_coordinates(row['unique_address'])
        if pd.isna(row['longitude']):
            row['longitude'] = longitude
        if pd.isna(row['latitude']):
            row['latitude'] = latitude
    return row

# Apply the function to fill missing values
mug = mug.apply(fill_missing_coordinates, axis=1)

In [24]:
# print rows with missing values in longitude and latitude column
pit[mug['longitude'].isnull() | mug['latitude'].isnull()]

/var/folders/cv/bbbq9ykj0jg6pt6c7tx5_vjh0000gn/T/ipykernel_74370/946519092.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  pit[mug['longitude'].isnull() | mug['latitude'].isnull()]


,unit,campus,province,region,ambusitecode,latitude,longitude,address,ZIP,unique_address


## AMB - generate longitude and latitude

In [25]:
amb.head()

,base,medical_resource,province,region,departure_location,departure_location_number,latitude,longitude,occasional_permanence
0,BA ANTW [Borgerhout] AMBUCE,ZW ANTWERPEN 7 (Team: AAANTW07A),Antwerpen,Vlaams Gewest,Gijselstraat 150 2140 BORGERHOUT,3811,51.215516,4.443788,N
1,BA ANTW [Hoboken] AMBUCE,ZW ANTWERPEN 9 (Team: AAANTW09A),Antwerpen,Vlaams Gewest,Antwerpsesteenweg 242 2660 HOBOKEN,3812,51.178831,4.361948,N
2,BA ANTW [Hof Ter Schelde] AC Antwerpen,ZW ANTWERPEN 3 (Team: AAANTW03A),Antwerpen,Vlaams Gewest,August Vermeylenlaan 6 2050 ANTWERPEN,10902,51.232810,4.377237,N
3,BA ANTW [Kaai 606] SIHWA,ZW ANTWERPEN HAVEN 1 (Team: AAAPOR01A)_x000D_Z...,Antwerpen,Vlaams Gewest,Antwerpsebaan 4 (Kaai 606) 2040 ANTWERPEN,12902,51.315516,4.321506,N
4,BA ANTW [Sint Elisabeth] AC Antwerpen,ZW ANTWERPEN 2 (Team: AAANTW02A),Antwerpen,Vlaams Gewest,Leopoldstraat 26 2000 ANTWERPEN,10901,51.213623,4.405718,N


In [26]:
amb[amb.duplicated(['latitude',	'longitude'])]

,base,medical_resource,province,region,departure_location,departure_location_number,latitude,longitude,occasional_permanence


In [27]:
# print rows with missing values in longitude and latitude column
pit[amb['longitude'].isnull() | amb['latitude'].isnull()]

/var/folders/cv/bbbq9ykj0jg6pt6c7tx5_vjh0000gn/T/ipykernel_74370/29285388.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  pit[amb['longitude'].isnull() | amb['latitude'].isnull()]


,unit,campus,province,region,ambusitecode,latitude,longitude,address,ZIP,unique_address


# conversion 
convert the datasets to geodataframe

In [29]:
import geopandas as gpd

In [30]:
# combine latitude and longitude to create a geometry column in amb
amb['geometry'] = gpd.points_from_xy(amb['longitude'], amb['latitude'])

# combine latitude and longitude to create a geometry column in amb
pit['geometry'] = gpd.points_from_xy(pit['longitude'], pit['latitude'])

# combine latitude and longitude to create a geometry column in amb
mug['geometry'] = gpd.points_from_xy(mug['longitude'], mug['latitude'])

# convert amb , mug, pit to geodataframe
amb = gpd.GeoDataFrame(amb, geometry='geometry')
pit = gpd.GeoDataFrame(pit, geometry='geometry')
mug = gpd.GeoDataFrame(mug, geometry='geometry')

## only keep necessary columns

In [31]:
amb = amb[['province', 'region', 'geometry']]
pit = pit[['province', 'region', 'geometry']]
mug = mug[['province', 'municipality', 'geometry']]

# rename province to province_amb in amb, region to region_amb in amb, geometry to geometry_amb in amb
amb.rename(columns={'province': 'province_amb', 'region': 'region_amb', 'geometry': 'geometry_amb'}, inplace=True)
pit.rename(columns={'province': 'province_pit', 'region': 'region_pit', 'geometry': 'geometry_pit'}, inplace=True)
mug.rename(columns={'province': 'province_mug', 'municipality': 'municipality_mug', 'geometry': 'geometry_mug'}, inplace=True)

In [32]:
amb = amb.set_geometry('geometry_amb')
pit = pit.set_geometry('geometry_pit')
mug = mug.set_geometry('geometry_mug')

In [33]:
amb.set_crs(epsg=4326, inplace=True)
pit.set_crs(epsg=4326, inplace=True)
mug.set_crs(epsg=4326, inplace=True)

,province_mug,municipality_mug,geometry_mug
0,Antwerpen,ANTWERPEN,POINT (4.41576 51.23138)
1,Antwerpen,ANTWERPEN,POINT (4.41251 51.20579)
2,Antwerpen,DEURNE,POINT (4.47078 51.20656)
3,Antwerpen,ANTWERPEN,POINT (4.98506 51.36387)
4,Antwerpen,WILRIJK,POINT (4.42009 51.17435)
...,...,...,...
89,Luik,VERVIERS,POINT (6.00713 50.71557)
90,Luxemburg,BASTOGNE,POINT (5.68312 50.15913)
91,Luxemburg,ARLON,POINT (5.82116 49.67801)
92,Luxemburg,LIBRAMONT,POINT (5.38007 49.92383)


# export 

In [38]:
# export the geodataframe to a shapefile
folder_path = "/Users/camillecu/Downloads/KUL/MDA/project/clustering/vector_shape"
amb.to_file(folder_path + "amb.shp")
pit.to_file(folder_path + "pit.shp")
mug.to_file(folder_path + "mug.shp")

/var/folders/cv/bbbq9ykj0jg6pt6c7tx5_vjh0000gn/T/ipykernel_74370/2780380701.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  amb.to_file(folder_path + "amb.shp")
/var/folders/cv/bbbq9ykj0jg6pt6c7tx5_vjh0000gn/T/ipykernel_74370/2780380701.py:4: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  pit.to_file(folder_path + "pit.shp")
/var/folders/cv/bbbq9ykj0jg6pt6c7tx5_vjh0000gn/T/ipykernel_74370/2780380701.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  mug.to_file(folder_path + "mug.shp")


In [34]:
# export pit to folder /Users/camillecu/Downloads/KUL/MDA/project/clustering
pit.to_csv('/Users/camillecu/Downloads/KUL/MDA/project/clustering/pit.csv', index=False)

In [35]:
mug.to_csv('/Users/camillecu/Downloads/KUL/MDA/project/clustering/mug.csv', index=False)

In [36]:
amb.to_csv('/Users/camillecu/Downloads/KUL/MDA/project/clustering/amb.csv', index=False)